In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Función para fuzzy matching de titulos

In [2]:
def title_matcher(title, title_list, thresh=75):
    matchs = process.extract(title, title_list, scorer=fuzz.token_sort_ratio)
    matchs = [m for m in matchs if m[1] >= thresh]
    if len(matchs) > 0 and matchs != []:
        matchs = matchs[0]
    else:
        matchs = ()
    return(matchs)

In [3]:
df = pd.read_csv('../Data/El_Recodo_letras.csv')
df.drop([' BPM', ' POPULARITY'], axis=1, inplace=True)
df.columns

Index(['AUTHOR', 'AUTHOR (2)', 'COMPOSER', 'DATE', 'DIRECTOR', 'LYRICS',
       'ORCHESTRA', 'SINGER', 'SOLOIST', 'TITLE', 'URL'],
      dtype='object')

In [4]:
pattern = re.compile('\((.*?)\)')
df['GENRE'] = df['TITLE'].str.findall(pattern).str.join('')
df['TITLE'] = df['TITLE'].str.replace(pattern, '')
df['TITLE'] = df['TITLE'].str.strip()
df['TITLE'] = df['TITLE'].str.lower()
df['TITLE'] = df['TITLE'].str.replace('á', 'a')
df['TITLE'] = df['TITLE'].str.replace('é', 'e')
df['TITLE'] = df['TITLE'].str.replace('í', 'i')
df['TITLE'] = df['TITLE'].str.replace('ó', 'o')
df['TITLE'] = df['TITLE'].str.replace('ú', 'u')

In [5]:
pattern = 'EL RECODO TANGO\n\t\t\tDesign Web: Gregory DIAZ'
df['LYRICS'] = [np.NAN if i == pattern else i for i in df['LYRICS']]

In [6]:
df['ano'] = df['DATE'].str.slice(0,4)

In [7]:
df.columns = ['autor', 'autor2', 'compositor', 'fecha', 'director', 'letra', 'orquesta', 'cantante', 'solista', 
              'titulo', 'link', 'ritmo', 'ano']

In [8]:
df_recodo = df.sort_values('fecha').groupby('titulo', as_index=False).first()
df_recodo.drop(['autor2', 'director', 'orquesta', 'cantante', 'solista'], axis=1, inplace=True)

## Match fechas

In [9]:
df_ttango = pd.read_csv('../Data/Todo_Tango_letras_final.csv')

In [10]:
df_recodo = df_recodo[~df_recodo.letra.isnull()]

In [11]:
df_merged = df_ttango.merge(df_recodo, on='titulo', how='left')

In [12]:
df_merged[(df_merged.ano_x == 'vacio')].shape

(7179, 14)

In [13]:
df_merged.shape

(10056, 14)

In [14]:
df_merged[(df_merged.ano_x == 'vacio') & ~(df_merged.ano_y.isnull())].shape

(1378, 14)

In [19]:
df_merged['letra_cons'] = df_merged['letra_x']

In [20]:
df_merged.loc[(df_merged.letra_x.isnull()) & (~df_merged.letra_y.isnull()), 'letra_cons'] = df_merged[(df_merged.letra_x.isnull()) & (~df_merged.letra_y.isnull())].letra_y

In [23]:
df_merged['letra_cons'].isnull().value_counts()

False    6260
True     3796
Name: letra_cons, dtype: int64

In [24]:
df_merged['letra_x'].isnull().value_counts()

False    5733
True     4323
Name: letra_x, dtype: int64

In [46]:
df_merged[df_merged['ano_x']=='vacio'].ano_x.value_counts()

vacio    7179
Name: ano_x, dtype: int64

In [37]:
df_merged['year_cons'] = df_merged['ano_x']

In [45]:
df_merged.loc[(df_merged.ano_x == 'vacio') & (~df_merged.ano_y.isnull()),'year_cons'] = df_merged.loc[(df_merged.ano_x == 'vacio') & (~df_merged.ano_y.isnull())].ano_y

In [47]:
df_merged[df_merged['year_cons']=='vacio'].ano_x.value_counts()

vacio    5801
Name: ano_x, dtype: int64

In [49]:
df_merged.columns = ['link_tt', 'titulo', 'ritmo_tt', 'ano_tt', 'musica_tt', 'compositor_tt',
       'letra_tt', 'autor_re', 'compositor_re', 'fecha_re', 'letra_re', 'link_re',
       'ritmo_re', 'ano_re', 'letra_cons', 'year_cons']

In [50]:
df_merged.to_csv('../Data/data_merged_raw', index=False, sep=";")

In [52]:
df_merged.drop(['ano_tt',  'letra_tt', 'letra_re', 'fecha_re', 'ritmo_re', 'ano_re'], axis=1, inplace=True)

In [53]:
df_merged.to_csv('../Data/data_merged_proc', index=False, sep=";")

In [ ]:
df_ttango_NA = df_ttango[df_ttango['ano'] == 'vacio']

In [ ]:
matched = df_ttango_NA.loc[0:10, 'titulo'].apply(title_matcher, title_list=df_recodo['TITLE'])

In [ ]:
matched

In [ ]:
matched2 = [t for t in matched if t != []]
titles = [t[0] for t in matched2 if t[1] > 90]
f1tt = [True if t in titles else False for t in df_ttango['titulo']]
f2tt = df_ttango['ano'] == 'vacio'
filtrec = [True if t in titles else False for t in df_recodo['TITLE']]

In [ ]:
df_ttango.loc[f1tt & f2tt, 'ano2'].shape

In [ ]:
df_recodo.loc[filtrec].DATE.shape

In [ ]:
df_recodo.loc[filtrec, ['TITLE','DATE']]

In [ ]:
df_ttango.loc[filttt, ['titulo', 'ano']]

In [ ]:
df_ttango2.ano.isnull().value_counts()

In [ ]:
df_ttango.ano.isnull().value_counts()

### Matcheamos letras

In [ ]:
df_ttango[df_ttango['titulo'] == matched].ano = df_recodo[df_recodo['TITLE'] == matched].DATE

In [ ]:
#matched = [i for i in matched if i != []]

In [ ]:
df[df['titulo'] == matched].ano = df_recodo[df_recodo['TITLE'] == matched].DATE